In [ ]:
import osmnx as ox
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
from tqdm import tqdm
import warnings
import time
import statistics as st

In [ ]:
from utilities import *
from simulation import traffic_simulation

In [ ]:
warnings.filterwarnings("ignore")
plt.rcParams['animation.embed_limit'] = 2**128

start_time e end_time serviranno alla fine per il tempo di computazione

In [ ]:
start_time = time.time()

In [ ]:
end_time = time.time()

print(f'time total = {end_time - start_time}s')

Latitudes and longitudes of 'Zona 30' cities except Minneapolis (all the italians except Bologna are in discussion)

In [ ]:
cities = {'Bologna' : (44.495555, 11.3428), 'Ferrara' : (44.835297, 11.619865), 'Modena' : (44.64582, 10.92572), 'Reggio Emilia' : (44.7, 10.633333), 
          'Firenze' : (43.771389, 11.254167), 'Torino' : (45.079167, 7.676111), 'Napoli' : (40.833333, 14.25), 'Milano' : (45.466944, 9.19), 
          'Roma' : (41.893056, 12.482778), 'Bergamo' : (45.695, 9.67), 'Verona' : (45.438611, 10.992778), 'Lecce' : (40.352011, 18.169139), 
          'Padova' : (45.406389, 11.877778), 'Olbia' : (40.916667, 9.5), 'Pesaro' : (43.91015, 12.9133), 'Lecco' : (45.85334, 9.39048),
          'Nantes' : (47.218056, -1.552778), 'Minneapolis' : (44.98, -93.268333), 'Bilbao' : (43.266667, -2.933334),'Zurich' : (47.374444, 8.541111), 
          'Valencia' : (39.483333, -0.366667), 'Barcelona' : (41.3825, 2.176944), 'Madrid' : (40.415524, -3.707488), 'Toronto' : (43.716589, -79.340686), 
          'Bruxelles' : (50.846667, 4.351667), 'Lyon' : (45.766944, 4.834167), 'Graz' : (47.070833, 15.438611), 'Edinburgh' : (55.953333, -3.189167)}

In [ ]:
city = 'Bologna'
city_radius = 3900 #in meters

# Create urban network from latitude and longitude
G = ox.graph_from_point(cities[city], dist=city_radius, dist_type='bbox', network_type='drive_service', simplify = True)
G = ox.project_graph(G)
osmids = list(G.nodes)
G = nx.relabel.convert_node_labels_to_integers(G)

# give each node its original osmid as attribute since we relabeled them
osmid_values = {k: v for k, v in zip(G.nodes, osmids)}
nx.set_node_attributes(G, osmid_values, "osmid")


'''fig, ax = ox.plot.plot_graph(G, bgcolor='black',node_size=5, node_color='white', show=False)
plt.axis('on')
plt.title(city)
plt.show()'''

#inizializing some useful graph attributes
clean_graph_data(G)

ztl = ['Tangenziale di Bologna', 'Autostrada Bologna-Padova','Autostrada Adriatica','Via Francesco Rizzoli','Via Ugo Bassi',"Via dell'Indipendenza"]
tangenziale = ['Tangenziale di Bologna', 'Autostrada Bologna-Padova','Autostrada Adriatica']
remove_edges(G,tangenziale)
fig, ax = ox.plot.plot_graph(G, bgcolor='black',node_size=5, node_color='white', show=False)
plt.axis('on')
plt.title(city)
plt.show()

Queste due sotto serviranno dopo per la Città50

In [ ]:
strade10 = []
limite10 = []
i=1
with open('strade10.txt','r') as file:
    for line in file:
    #lines = [line.strip() for line in file.readlines()]
        #values = line.split()
        if i%2 != 0:
            lines = line.strip()
            strade10.append(lines)
        else: 
            limite10.append(float(line))
        i+=1

In [ ]:
def città10(graph,streets,limits):
    for _,_,data in graph.edges(data=True):
        if 'name' in data:
            if data['name'] in streets:
                index = streets.index(data['name'])
                data['maxspeed'] = limits[index]

In [ ]:
connectivity_analysis(G,city)


In [ ]:
degree_histogram(G,city)

In [ ]:
centrality_analysis(G,city)

In [ ]:
#computing Global Efficiency
G1 = global_efficiency(G)

#computing Information Centrality
G.remove_edge(4,7) #remove some edges wiht for loops
G2 = global_efficiency(G) #ma perchè scriviamo in inglese?
print(G1,G2, G1-G2, (G1-G2)/G1) #scemo chi legge
#G.edges #scemo chi l'edges

In [ ]:
città50(G)

In [ ]:
closeness_centrality = nx.closeness_centrality(G)

In [ ]:
n_frame, dt, speed_up, n_cars= 720, 0.02, 500, 3000

cars_velocities , cars_accelerations, max_accel_per_frame, max_velocity_per_frame, mean_speed, mean_accel, travelling_times, mean_travelling_time, street_passage, accident_positions = traffic_simulation(G,
                n_cars = n_cars,
                radius = city_radius,
                speed_up = speed_up,
                closeness_centrality=closeness_centrality,
                size = 2*city_radius,
                scale = 0.3,
                dt = dt,
                n_frame = n_frame, 
                animate=False)

In [ ]:
if max(max_velocity_per_frame) < 100:
    already_passed = []
    edge_counts = []
    with open('edges_passage.txt','r') as file:
        for line in file:
            values = line.split()
            # Convert the values to integers (assuming they're integers)
            edge1 = int(values[0])
            edge2 = int(values[1])
            count = int(values[2])
            already_passed.append((edge1,edge2))
            edge_counts.append(count)


    for edge in street_passage:
        if (edge[0],edge[1]) in already_passed:
            index = already_passed.index((edge[0],edge[1]))
            edge_counts[index] += 1
            continue
        already_passed.append((edge[0],edge[1]))
        edge_counts.append(edge[2])
            
    with open('edges_passage.txt','w') as file:
        pass

    with open('edges_passage.txt','w') as file:
        for edge in already_passed:
            index = already_passed.index(edge)
            file.write(f"{edge[0]} {edge[1]} {edge_counts[index]}\n")

    # travelling times and mean speed
    with open('travelling_times.txt','a') as file:
        for time in travelling_times:
            file.write(f"{time}\n")

    with open('mean_speed.txt','a') as file:
        file.write(f"{mean_speed}\n")
    
    with open('mean_accel.txt','a') as file:
        file.write(f"{mean_accel}\n")

    #accident positions
    if accident_positions:
        with open('accident_position.txt','a') as file:
            for accident in accident_positions:
                file.write(f"{accident[0]} {accident[1]}\n")
    print('Alright my man...')

In [ ]:
mean_velocity_per_frame = np.zeros(np.shape(cars_velocities)[0])
mean = 0
for i in range(np.shape(cars_velocities)[0]):
    mean_velocity_per_frame[i] = np.mean(cars_velocities[i])
    print(np.mean(cars_velocities[i]))
    
#mean = np.sum(mean_velocity_per_frame)
#mean

In [ ]:
max(max_velocity_per_frame)

In [ ]:
#DON'T RUN!!!!!!!!!
#DON'T
#RUN!!!!!!!!!!!
with open('edges_passage.txt','w') as file:
    pass

with open('mean_speed.txt','w') as file:
    pass

with open('mean_accel.txt','w') as file:
    pass

with open('travelling_times.txt','w') as file:
    pass

with open('accident_position.txt','w') as file:
    pass

## Writing on a txt file the edge passage

In [ ]:
already_passed = []
edge_counts = []
with open('edges_passage.txt','r') as file:
    for line in file:
        values = line.split()
        # Convert the values to integers (assuming they're integers)
        edge1 = int(values[0])
        edge2 = int(values[1])
        count = int(values[2])
        already_passed.append((edge1,edge2))
        edge_counts.append(count)


for edge in street_passage:
    if (edge[0],edge[1]) in already_passed:
        index = already_passed.index((edge[0],edge[1]))
        edge_counts[index] += 1
        continue
    already_passed.append((edge[0],edge[1]))
    edge_counts.append(edge[2])
        
with open('edges_passage.txt','w') as file:
    pass

with open('edges_passage.txt','w') as file:
    for edge in already_passed:
        index = already_passed.index(edge)
        file.write(f"{edge[0]} {edge[1]} {edge_counts[index]}\n")

# travelling times and mean speed
with open('travelling_times.txt','a') as file:
    for time in travelling_times:
        file.write(f"{time}\n")

with open('mean_speed.txt','a') as file:
    file.write(f"{mean_speed}\n")

with open('mean_accel.txt','a') as file:
    file.write(f"{mean_accel}\n")

#accident positions
if accident_positions:
    with open('accident_position.txt','a') as file:
        for accident in accident_positions:
            file.write(f"{accident[0]} {accident[1]}\n")

In [ ]:
len(already_passed)

## Reading from a txt file

In [ ]:
streets = []
with open('edges_passage.txt','r') as file:
    for line in file:
        values = line.split()
        # Convert the values to integers (assuming they're integers)
        edge1 = int(values[0])
        edge2 = int(values[1])
        count = int(values[2])
        streets.append((edge1,edge2,count))


Travelling_times = []
with open('travelling_times.txt','r') as file:
    for line in file:
        values = line.split()
        # Convert the values to integers (assuming they're integers)
        time = float(values[0])
        Travelling_times.append(time)

Travelling_times = np.array(Travelling_times)
# Create a figure and axis for plotting
fig, ax = plt.subplots()

# Define the initial histogram
n_bins = int(max(Travelling_times/60))
hist, _ = np.histogram(Travelling_times/60, bins=n_bins)
bars = ax.bar(range(n_bins), hist)

ax.set_title('Travelling times')
ax.set_xlabel('minutes')
ax.set_ylabel('Frequency')

mean_travelling_time = sum(Travelling_times)/len(Travelling_times)
mode = st.mode(Travelling_times)

plt.text(0.95, 0.95, f"Mean: {int(mean_travelling_time/60)}m {(mean_travelling_time/60 - int(mean_travelling_time/60))*60:.2f}s\n Mode: {int(mode/60)}m {(mode/60 - int(mode/60))*60:.2f}s\n", 
         horizontalalignment='right', verticalalignment='top', transform=plt.gca().transAxes)
#print(f'Mode: {int(mode/60)}m {(mode/60 - int(mode/60))*60:.2f}s\n')
#print(f'Mean travelling time: {int(mean_travelling_time/60)}m {round((mean_travelling_time/60 - int(mean_travelling_time/60))*60,2)}s\n')

plt.show()

In [ ]:
show_traffic_and_accident(G,street_passage,accident_positions)

### Animazione velocità 

In [ ]:
animate_speeds(cars_velocities,max_velocity_per_frame,n_cars,speed_up,dt)

Acceleration animation (?)

In [ ]:
animate_accelerations(cars_velocities,max_velocity_per_frame,n_cars,speed_up,dt)

# DATI TRAFFICO REALI

In [ ]:
traffico = pd.read_excel('rilevazione-flusso-veicoli-tramite-spire-anno-2024.xlsx')

In [ ]:
traffico['total'] = np.zeros(len(traffico['x']))

In [ ]:
traffico

In [ ]:
for dati in traffico:
    if dati == '05:00-06:00':
        print(dati)
a = 11
b = 13
a_s = str(11)
b_s = str(13)
d = b-a
for _ in range(d):
    print(a_s+':00-'+str(int(a_s) + 1)+':00')
    a += 1
    a_s = str(a)


#traffico[a+':00-'+str(int(a) + 1)+':00']
#traffico['11:00-12:00']


# CAMBIA IL NUMERO DI 'PASSAGE'

In [ ]:
def show_traffic_data(graph,traffic_data,days,hours = [0,24]): 
    traffic_edges = []
    passage_values = []
    
    #data = traffic_data['latitudine'],traffic_data['longitudine'],traffic_data['giorno settimana']

    # Filter traffic data based on 'giorno settimana'
    filtered_data = traffic_data[traffic_data['giorno settimana'].isin(days)]
    min_hour = hours[0]
    max_hour = hours[1]
    diff_hours = max_hour - min_hour
    min_hour_str = str(min_hour)
    max_hour_str = str(max_hour)
    d = max_hour - min_hour
    for _ in range(d):
        filtered_data['total'] += filtered_data[min_hour_str+':00-'+str(int(min_hour_str) + 1)+':00']
        min_hour += 1
        min_hour_str = str(min_hour)

    # Use `nearest_edges` function on filtered data
    #traffic_edges = filtered_data.apply(lambda row: ox.nearest_edges(graph, row['latitudine'], row['longitudine'])[:2], axis=1)

    '''with tqdm(total=len(filtered_data)) as pbar:
        # Use apply function to apply nearest_edges function on each row
        traffic_edges = filtered_data.apply(lambda row: ox.nearest_edges(graph, row['latitudine'], row['longitudine'])[:2], axis=1)
        pbar.update(len(filtered_data))'''
    # Calculate the total number of rows in the DataFrame
    #total_rows = len(filtered_data)

    traffic_edges = ox.nearest_edges(graph,filtered_data['x'],filtered_data['y'])

    for a in filtered_data['total']:
        passage_values.append(a)

    
    #with tqdm(total=total_rows) as pbar:
        #for lat, long in zip(filtered_data['latitudine'], filtered_data['longitudine']):
            #traffic_edges.append(ox.nearest_edges(graph,lat,long)[:2])

            #pbar.update(1)

    #for lat,long in tqdm(zip(filtered_data['latitudine'],filtered_data['longitudine'])):
        #if filtered_data['giorno settimana'][i] in days:
        #traffic_edges.append(ox.nearest_edges(graph,data[0][i],data[1][i])[:2])
        #traffic_edges.append(ox.nearest_edges(graph,lat,long)[:2])

    nodes,edges = ox.graph_to_gdfs(graph,nodes=True,edges=True)

    for i in range(len(traffic_edges)):
        edges.loc[(traffic_edges[i][0],traffic_edges[i][1]),'passage'][0] = passage_values[i]
    

    min_passage = sum(passage_values)/len(passage_values)
    
    for u,v,data in graph.edges(data=True):
        if data['passage'] == 0:
            edges.loc[(u,v),'passage'][0] += 0
            
    #for edge in traffic_edges:
        #edges.loc[(edge[0],edge[1]),'passage'][0] += edges[2]

    graph = ox.graph_from_gdfs(nodes,edges)

    #Get edge passages
    edge_values = np.array([data['passage'] for u, v, data in graph.edges(data=True)])
    norm_edge_values = (edge_values - min(edge_values))/(max(edge_values - min(edge_values)))

    # Define a colormap
    cmap = plt.cm.RdBu 

    # Normalize edge lengths
    norm = plt.Normalize(min(edge_values), max(edge_values))

    # Map edge lengths to colors
    colors = [cmap(norm(value)) for value in edge_values]

    #Plot the network with colored edges
    fig , ax = ox.plot_graph(graph, edge_color=plt.cm.RdBu(norm_edge_values), node_size=0, bgcolor='black', edge_linewidth=0.5,show=False)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, ax=ax,label='Edge Passage')
    plt.title('Traffic data')
    plt.show()

    return passage_values
    

In [ ]:
days = ['Mercoledì']
passage_values = show_traffic_data(G,traffico,days,[11,13])

In [ ]:
'''for _,_,data in G.edges(data=True):
        if float(data['maxspeed']) > 40:
            data['maxspeed'] = 0.001'''
# Get edge passages
edge_values = np.array([data['maxspeed'] for u, v, key, data in G.edges(keys=True, data=True)])
norm_edge_values = (edge_values - min(edge_values))/(max(edge_values - min(edge_values)))

# Define a colormap
cmap = plt.cm.RdBu 

# Normalize edge lengths
norm = plt.Normalize(min(edge_values), max(edge_values))

# Map edge lengths to colors
colors = [cmap(norm(value)) for value in edge_values]

#Plot the network with colored edges
fig , ax = ox.plot_graph(G, edge_color=colors, node_size=0, bgcolor='black', edge_linewidth=0.5,show=False)

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, ax=ax,label='Edge Passage')
plt.title('Roads more populated')
plt.show()

# Incidenti

In [ ]:
incidenti = pd.read_excel('Incidenti.xlsx')

In [ ]:
incidenti

In [ ]:
print(incidenti['DATA'][0].year,type(incidenti['DATA'][0].year))
print(type(incidenti['GIORNO'][0]))
print(incidenti['ORA'][0],type(incidenti['ORA'][0]))
print(type(incidenti['X'][1].astype(float)))
len(incidenti['X'])
a = [2010,2011]
incidenti['DATA'][0].year in a

In [ ]:
def show_accident_data(graph,accident_data,years,hours=[0,24]):
    nodes,edges = ox.graph_to_gdfs(graph,nodes=True,edges=True)
    
    graph = ox.graph_from_gdfs(nodes,edges)
    
    edge_accident = []
    hour = accident_data['ORA'].astype(float)

    x = accident_data['X'].astype(float)
    y = accident_data['Y'].astype(float)


    
    for i in range(len(accident_data)):
        if accident_data['DATA'][i].year in years: #and hour[i] > hours[0] and hour[i] < hours[1]:
            edge_accident.append(ox.nearest_edges(graph,x[i],y[i]))

    for edge in edge_accident:
        edges.loc[(edge[0],edge[1]),'accident'][0] += 1
    

    graph = ox.graph_from_gdfs(nodes,edges)

    #Get edge passages
    edge_values = np.array([data['accident'] for u, v, data in graph.edges(data=True)])
    norm_edge_values = (edge_values - min(edge_values))/(max(edge_values)-min(edge_values))
    # Define a colormap
    cmap = plt.cm.RdBu 

    # Normalize edge lengths
    norm = plt.Normalize(min(edge_values), max(edge_values))

    # Map edge lengths to colors
    colors = [cmap(norm(value)) for value in edge_values]

    #Plot the network with colored edges
    fig , ax = ox.plot_graph(graph, edge_color=plt.cm.RdBu(norm_edge_values), node_size=0., bgcolor='black', edge_linewidth=0.5,show=False)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, ax=ax,label='Edge Passage')
    plt.title('Traffic data')
    plt.show()


    


In [ ]:
show_accident_data(G,incidenti,[2018,2019,2020])

In [ ]:
x=incidenti['X']
y=incidenti['Y']

plt.scatter(x,y,s=0.5)
plt.show()